# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: wenshan wang
Date: 8/17

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [5]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
from pathlib import Path
env_path = Path(".") / ".env"
content = "ALPHAVANTAGE_API_KEY=zxcvbnm\n"
env_path.write_text(content, encoding="utf-8")
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [6]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [9]:
import sys, subprocess
print("Python:", sys.executable)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "yfinance"])


Python: /opt/miniconda3/bin/python
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 14.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.1 MB/s eta 0:00:00a 0:00:01m


  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334


  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15636 sha256=5966b6671c424fa1bb949244455b4871e9a4bb6f0ca52124b370a73c960c5ac9
  Stored in directory: /Users/wenshan/Library/Caches/pip/wheels/1e/df/0f/e2bbb22d689b30c681feb5410ab64a2523437b34c8ecfc6476
  Created wheel for peewee: filename=peewee-3.18.2-cp313-cp313-macosx_11_0_arm64.whl size=262659 sha256=979829df2f678603f79dd592b544bb93dfcd0aef986b8c2052192004dedfdeef
  Stored in directory: /Users/wenshan/Library/Caches/pip/wheels/1c/48/cc/00b7d0e7defa21a58915654917c89eaedd32a6e614d8e4ad92
Successfully built multitasking peewee
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [yfinance]5/6 [yfinance]


0

In [10]:
import yfinance as yf
print("yfinance version:", yf.__version__)

yfinance version: 0.2.65


In [30]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))

import pandas as pd

if USE_ALPHA:
    import requests, os
    url = 'https://www.alphavantage.co/query'
    params = {
        'function':'TIME_SERIES_DAILY_ADJUSTED',
        'symbol':SYMBOL,
        'outputsize':'compact',
        'apikey':os.getenv('ALPHAVANTAGE_API_KEY')
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()

    keys = [k for k in js if 'Time Series' in k]
    if not keys:
        print("Alpha Vantage not usable →", js.get('Note') or js.get('Error Message') or f"Keys returned: {list(js.keys())}")
        print("Falling back to yfinance…")
        use_yf = True
    else:
        key = keys[0]
        df_api = (
            pd.DataFrame(js[key]).T
            .reset_index()
            .rename(columns={'index':'date','5. adjusted close':'adj_close'})
            [['date','adj_close']]
        )
        df_api['date'] = pd.to_datetime(df_api['date'], errors='coerce')
        df_api['adj_close'] = pd.to_numeric(df_api['adj_close'], errors='coerce')
        source_used = 'alpha'
        use_yf = False
else:
    print("No ALPHAVANTAGE_API_KEY detected → using yfinance.")
    use_yf = True

if use_yf:
    import yfinance as yf
    # Force auto_adjust=False so 'Adj Close' exists
    df = yf.download(SYMBOL, period='3mo', interval='1d', auto_adjust=False, progress=False)

    # 1) If columns are MultiIndex like [('Adj Close','AAPL'), ...], collapse to first level
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [c[0] for c in df.columns]  # keep 'Adj Close','Close','High','Low','Open','Volume'

    # 2) Make date a column
    df = df.reset_index()

    # 3) After reset_index(), sometimes first col becomes ('Date',''); normalize all tuple cols
    df.columns = [c[0] if isinstance(c, tuple) else c for c in df.columns]

    # 4) Choose date and price columns robustly
    date_col = 'Date' if 'Date' in df.columns else ('date' if 'date' in df.columns else ('index' if 'index' in df.columns else None))
    price_col = 'Adj Close' if 'Adj Close' in df.columns else ('Close' if 'Close' in df.columns else None)

    if date_col is None or price_col is None:
        print("DEBUG yfinance columns:", list(df.columns))
        raise KeyError("Could not locate date or price columns in yfinance result.")

    df_api = df[[date_col, price_col]].rename(columns={date_col:'date', price_col:'adj_close'})
    df_api['date'] = pd.to_datetime(df_api['date'], errors='coerce')
    df_api['adj_close'] = pd.to_numeric(df_api['adj_close'], errors='coerce')
    source_used = 'yfinance'

# Unchanged validator call from your notebook
v_api = validate(df_api, ['date','adj_close'])

print("Source used:", source_used)
print("Validation:", v_api)
df_api.head()


Alpha Vantage not usable → Keys returned: ['Information']
Falling back to yfinance…
Source used: yfinance
Validation: {'missing': [], 'shape': (63, 2), 'na_total': 0}


,date,adj_close
0,2025-05-16,211.020508
1,2025-05-19,208.543320
2,2025-05-20,206.625504
3,2025-05-21,201.860901
4,2025-05-22,201.131729


In [31]:
_ = save_csv(
    df_api.sort_values('date'),
    prefix='api',
    source='yfinance',
    symbol=SYMBOL
)


Saved data/raw/api_source-yfinance_symbol-AAPL_20250818-011514.csv


In [32]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved data/raw/api_source-alpha_symbol-AAPL_20250818-011602.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [37]:
import sys, subprocess
print("Python:", sys.executable)

# Install one of the parsers pandas.read_html needs
subprocess.check_call([sys.executable, "-m", "pip", "install", "lxml", "beautifulsoup4", "html5lib"])


Python: /opt/miniconda3/bin/python
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 2.1 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [html5lib]1/2 [html5lib]


0

In [ ]:
import pandas as pd

URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

tables = pd.read_html(URL)
print("Number of tables found:", len(tables))

candidates = [t for t in tables if t.shape[1] >= 5 and t.shape[0] > 50]
df_web_raw = candidates[0] if candidates else tables[0]

print("Parsed table shape:", df_web_raw.shape)
print("Columns:", list(df_web_raw.columns))

rename_map = {
    'Symbol': 'symbol',
    'Security': 'security',
    'GICS Sector': 'sector',
    'GICS sector': 'sector',
    'Headquarters Location': 'hq',
    'Date first added': 'date_added',
}
cols_available = [c for c in df_web_raw.columns if c in rename_map]
df_web = df_web_raw[cols_available].rename(columns=rename_map)

if 'date_added' in df_web.columns:
    df_web['date_added'] = pd.to_datetime(df_web['date_added'], errors='coerce')

required_cols = [c for c in ['symbol','security'] if c in df_web.columns]
v_web = validate(df_web, required_cols) if required_cols else {'missing': ['symbol','security'], 'shape': df_web.shape, 'na_total': None}

print("\nPreview (top 5 rows):")
display(df_web.head())

print("\nValidation:", v_web)


Number of tables found: 2
Parsed table shape: (503, 8)
Columns: ['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded']

Preview (top 5 rows):


,symbol,security,sector,hq
0,MMM,3M,Industrials,"Saint Paul, Minnesota"
1,AOS,A. O. Smith,Industrials,"Milwaukee, Wisconsin"
2,ABT,Abbott Laboratories,Health Care,"North Chicago, Illinois"
3,ABBV,AbbVie,Health Care,"North Chicago, Illinois"
4,ACN,Accenture,Information Technology,"Dublin, Ireland"



Validation: {'missing': [], 'shape': (503, 4), 'na_total': 0}


In [40]:

keep_cols = [c for c in ['symbol','security','sector','hq','date_added'] if c in df_web.columns]
df_save = df_web[keep_cols].copy()

if 'symbol' in df_save.columns:
    df_save['symbol'] = df_save['symbol'].astype(str).str.strip().str.upper()
if 'security' in df_save.columns:
    df_save['security'] = df_save['security'].astype(str).str.strip()
if 'sector' in df_save.columns:
    df_save['sector'] = df_save['sector'].astype(str).str.strip()
if 'hq' in df_save.columns:
    df_save['hq'] = df_save['hq'].astype(str).str.strip()

_ = save_csv(
    df_save,
    prefix='web',
    source='wikipedia',
    table='sp500'
)


Saved data/raw/web_source-wikipedia_table-sp500_20250818-012150.csv


In [42]:
try:
    SYMBOL
except NameError:
    SYMBOL = "AAPL" 

if 'df_web' not in globals():
    raise RuntimeError("df_web is missing. Re-run the Part 2 scraping cell.")

in_sp500 = SYMBOL in set(df_web['symbol'].astype(str).str.upper())
print(f"Ticker {SYMBOL!r} found in scraped S&P 500 table? →", in_sp500)

df_lookup = df_web[df_web['symbol'].str.upper() == SYMBOL].copy()
print("Lookup shape:", df_lookup.shape)

display(df_lookup.head(3))


Ticker 'AAPL' found in scraped S&P 500 table? → True
Lookup shape: (1, 4)


,symbol,security,sector,hq
38,AAPL,Apple Inc.,Information Technology,"Cupertino, California"


## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.